<a href="https://colab.research.google.com/github/DCajiao/Time-series-forecast-of-energy-consumption-in-Tetouan-City/blob/main/notebooks/04_RNN_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Entrenamiento, Predicción y Evaluación de un modelo RNN

In [2]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

from datetime import datetime
from io import BytesIO

DATA_GITHUB_URL = 'https://raw.githubusercontent.com/DCajiao/Time-series-forecast-of-energy-consumption-in-Tetouan-City/refs/heads/main/data/zone1_power_consumption_of_tetouan_city.csv'

# Descargar los datos desde github
response = requests.get(DATA_GITHUB_URL)

# Convertir en un df desde el xlsx de github
df = pd.read_csv(BytesIO(response.content), sep=',')

In [3]:
df["datetime"] = pd.to_datetime(df["datetime"])
df = df.set_index("datetime")

# Ahora solo quedan las columnas numéricas
df.head()

,temperature,humidity,wind_speed,general_diffuse_flows,zone_1
datetime,,,,,
2017-01-01 00:00:00,6.559,73.8,0.083,0.051,34055.69620
2017-01-01 00:10:00,6.414,74.5,0.083,0.070,29814.68354
2017-01-01 00:20:00,6.313,74.5,0.080,0.062,29128.10127
2017-01-01 00:30:00,6.121,75.0,0.083,0.091,28228.86076
2017-01-01 00:40:00,5.921,75.7,0.081,0.048,27335.69620


In [4]:
# Evaluación
def smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(np.abs(y_true - y_pred) / denominator) * 100

def wape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true)) * 100

#### **Entrenamiento**

In [5]:
# Índices por % para train, val, test
n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

print(f"Número de características (features): {num_features}")

Número de características (features): 5


In [6]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [7]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):

        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                          enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(train_df.columns)}

        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]

        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,)
        ds = ds.map(self.split_window)
        return ds

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        result = getattr(self, '_example', None)
        if result is None:
            result = next(iter(self.train))
            self._example = result
        return result

In [8]:
# Un solo paso (10 minutos hacia adelante)
single_step_window = WindowGenerator(
    input_width=6480,  # 45 días de input
    label_width=1,
    shift=1,           # predecir 1 paso adelante
    label_columns=["zone_1"]
)

# Multi-step (3 horas adelante = 18 pasos de 10 min)
multi_step_window = WindowGenerator(
    input_width=6480,
    label_width=18,    # 18 pasos (3h)
    shift=18,          # salto de 3h
    label_columns=["zone_1"]
)


In [9]:
def compile_and_fit(model, window, patience=2, epochs=20):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min')
    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=[tf.keras.metrics.MeanAbsoluteError()])
    history = model.fit(window.train, validation_data=window.val,
                        epochs=epochs, callbacks=[early_stopping])
    return history

---

#### **Predicción**

A un paso

In [10]:
# Predicción 1 paso (10 min hacia adelante)
simple_rnn_single = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(20, return_sequences=True), # Estado interno
    tf.keras.layers.Dense(units=1)                        # salida = 1 paso
])

history_single = compile_and_fit(simple_rnn_single, single_step_window)

val_single = simple_rnn_single.evaluate(single_step_window.val, return_dict=True)
test_single = simple_rnn_single.evaluate(single_step_window.test, return_dict=True)

print("SimpleRNN Single-step Validation:", val_single)
print("SimpleRNN Single-step Test:", test_single)

Epoch 1/20
945/945 ━━━━━━━━━━━━━━━━━━━━ 352s 368ms/step - loss: 0.9364 - mean_absolute_error: 0.8042 - val_loss: 1.1167 - val_mean_absolute_error: 0.8960
Epoch 2/20
260/945 ━━━━━━━━━━━━━━━━━━━━ 3:59 350ms/step - loss: 0.9073 - mean_absolute_error: 0.7915

KeyboardInterrupt: 

A múltiples pasos

In [ ]:
simple_rnn_multi = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(64, return_sequences=False), # solo último estado
    tf.keras.layers.Dense(18),                             # salida = 18 pasos
    tf.keras.layers.Reshape([18, 1])                       # salida (batch, 18, 1)
])

history_multi = compile_and_fit(simple_rnn_multi, multi_step_window)

val_multi = simple_rnn_multi.evaluate(multi_step_window.val, return_dict=True)
test_multi = simple_rnn_multi.evaluate(multi_step_window.test, return_dict=True)

print("SimpleRNN Multi-step Validation:", val_multi)
print("SimpleRNN Multi-step Test:", test_multi)

---

#### **Analisis de resultados**

Métricas a evaluar:
1. sMAPE
2. WAPE

In [ ]:
y_true, y_pred = [], []
for x, y in single_step_window.test.take(20):
    preds = simple_rnn_single.predict(x)
    y_true.extend(y.numpy().reshape(-1))
    y_pred.extend(preds.reshape(-1))

print("Single-step SMAPE:", smape(y_true, y_pred))
print("Single-step WAPE:", wape(y_true, y_pred))


y_true, y_pred = [], []
for x, y in multi_step_window.test.take(20):
    preds = simple_rnn_multi.predict(x)
    y_true.extend(y.numpy().reshape(-1))
    y_pred.extend(preds.reshape(-1))

print("Multi-step SMAPE:", smape(y_true, y_pred))
print("Multi-step WAPE:", wape(y_true, y_pred))

---